- !!! COMBINE lr1, lr2 and lr4 || lr1 and lr4 || lr2 and lr4 get high correlation in lr with lr: 0.0001 - 0.01, batch: 10 - 120
- dr--,batch++ && batch+,dr+ && batch+ --> batch_size: high correlation

In [14]:
import pandas as pd 

path = 'reg_data/final_reg_data.csv'
df = pd.read_csv(path)
df = df[['batch_size', 'learning_rate','dropout', 'val_loss']].dropna()
df.corr()

,batch_size,learning_rate,dropout,val_loss
batch_size,1.000000,-0.008216,0.217588,-0.039265
learning_rate,-0.008216,1.000000,0.601031,0.769619
dropout,0.217588,0.601031,1.000000,0.605181
val_loss,-0.039265,0.769619,0.605181,1.000000


In [22]:
min_df = df[df['val_loss'] < 0.04]
min_df

,batch_size,learning_rate,dropout,val_loss
42,27,0.004962,0.210051,0.025696
44,42,0.002054,0.225468,0.034810


In [23]:
import plotly.express as px 
import plotly.graph_objects as go 

fig = go.Figure()
fig.add_traces(
    go.Mesh3d(
        x=df['learning_rate'], y=df['dropout'], z=df['val_loss'], 
        opacity=1,
        intensity=df['batch_size'], colorbar_title='batch_size',
        colorscale='viridis'
        )
    )
fig.add_traces(
    go.Scatter3d(
        x=min_df['learning_rate'], y=min_df['dropout'], z=min_df['val_loss']
    )
)
fig.update_layout(
    scene=dict(
        xaxis_title='learning_rate',
        yaxis_title='dropout',
        zaxis_title='val_loss',
    )
)
fig.show()

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

degree = 2
poly_reg = PolynomialFeatures(degree=degree)

x = df[['learning_rate', 'dropout', 'batch_size']]
y = df['val_loss']
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1)

poly_x_train = poly_reg.fit_transform(x_train)

In [29]:
regr = LinearRegression()
regr.fit(poly_x_train, y_train)
print(regr.score(poly_x_train, y_train))
print(regr.coef_)

0.7978715814575126
[ 0.00000000e+00  3.67193833e+00 -1.02315403e-01  1.15069971e-03
 -1.73101215e+02  2.13080950e+01 -8.29589437e-02  2.92295128e-02
  1.01998224e-03 -1.15761405e-05]


In [34]:
import numpy as np

poly_x_val = poly_reg.fit_transform(x_val)
poly_y_val = regr.predict(poly_x_val)
print("Mean absolute error: ", mean_squared_error(y_val, poly_y_val))


Mean absolute error:  0.006364667980726276


In [28]:
import statsmodels.api as sm 

x = sm.add_constant(x)
regr = sm.OLS(y,x).fit()
print(regr.summary())

                            OLS Regression Results                            
Dep. Variable:               val_loss   R-squared:                       0.632
Model:                            OLS   Adj. R-squared:                  0.622
Method:                 Least Squares   F-statistic:                     66.85
Date:                Fri, 20 Jan 2023   Prob (F-statistic):           2.98e-25
Time:                        15:55:07   Log-Likelihood:                 197.73
No. Observations:                 121   AIC:                            -387.5
Df Residuals:                     117   BIC:                            -376.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0242      0.015      1.640